In [2]:
!python -m pip install git+https://github.com/huggingface/transformers

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-xopx8mcb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-xopx8mcb
  Resolved https://github.com/huggingface/transformers to commit ca03842cdcf2823301171ab27aec4b6b1cafdbc1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
pip install datasets

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/d7/84/0df6c5981f5fc722381662ff8cfbdf8aad64bec875f75d80b55bfef394ce/datasets-3.2.0-py3-none-any.whl.metadata
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Obtaining dependency information for pyarrow>=15.0.0 from https://files.pythonhosted.org/packages/6d/84/8037c20005ccc7b869726465be0957bd9c29cfc88612962030f08292ad06/pyarrow-18.1.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata
  Using cached pyarrow-18.1.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Obtaining dependency information for requests>=2.32.2 from https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl.metadata
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Obtaining dependency information for t

In [1]:
pip install accelerate>=0.26.0

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pillow

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Note: you may need to restart the kernel to use updated packages.


In [12]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from PIL import Image
import time
import pandas as pd
import os

# Step 1: Load the processor and model
print("Loading processor and model...")

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", 
    trust_remote_code=True, 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True
)
model.to("cuda:0")

print("Processor and model loaded successfully.")

# Step 2: Load dataset from CSV
csv_path = "dataset.csv"  # Path to your CSV file
dataset = pd.read_csv(csv_path)

if 'image' not in dataset.columns or 'questions' not in dataset.columns:
    raise ValueError("CSV must contain 'image' and 'questions' columns.")

results = []

# Step 3: Process each image and its questions
for idx, row in dataset.iterrows():
    image_path = row['image']
    questions = row['questions'].split(';')  # Split questions by semicolon
    groundtruth_answers =row ['Groundtruth_answers']
    
    # Check if the image file exists
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        continue
    
    print(f"Processing image {image_path}...")
    image = Image.open(image_path)

    answers = []
    start_time = time.time()

    for question in questions:
        # Format the conversation for each question
        conversation = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question},
                    {"type": "image"}
                ]
            }
        ]
        prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

        # Prepare inputs for the model
        inputs = processor(images=image, text=prompt, return_tensors="pt").to("cuda:0")

        # Generate response
        output = model.generate(**inputs, max_new_tokens=400)
        decoded_output = processor.decode(output[0], skip_special_tokens=True)

        answers.append(decoded_output)  # Collect the answer

    end_time = time.time()

    # Append results
    results.append({
        "image": image_path,
        "questions": ";".join(questions),
        "generated_answers": ";".join(answers),
        "groundtruth_answers": groundtruth_answers
    })

# Step 4: Save results to a new CSV file
output_csv_path = "qwen2vl_generated_answers.csv"
results_df = pd.DataFrame(results)
results_df.to_csv(output_csv_path, index=False)

print(f"Processing complete. Results saved to {output_csv_path}.")


Loading processor and model...


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Processor and model loaded successfully.
Processing image datasett/datasetUML1.png...
Processing image datasett/datasetUML1.png...
Processing image datasett/datasetUML1.png...
Processing image datasett/datasetUML1.png...
Processing image datasett/datasetUML1.png...
Processing image datasett/datasetUML2.png...
Processing image datasett/datasetUML2.png...
Processing image datasett/datasetUML2.png...
Processing image datasett/datasetUML2.png...
Processing image datasett/datasetUML2.png...
Processing image datasett/datasetUML3.png...
Processing image datasett/datasetUML3.png...
Processing image datasett/datasetUML3.png...
Processing image datasett/datasetUML3.png...
Processing image datasett/datasetUML3.png...
Processing image datasett/datasetUML4.png...
Processing image datasett/datasetUML4.png...
Processing image datasett/datasetUML4.png...
Processing image datasett/datasetUML4.png...
Processing image datasett/datasetUML4.png...
Processing image datasett/datasetUML5.png...
Processing ima